**Лекция 4: PLSQL - Триггеры, Процедуры и Функции**

## Введение в PLSQL
PLSQL (Procedural Language/Structured Query Language) — это процедурное расширение SQL, разработанное компанией Oracle. Оно позволяет писать сложные программные конструкции, включая триггеры, процедуры и функции, что делает работу с базой данных более гибкой и мощной. 

Использование PLSQL особенно важно в ситуациях, когда требуется автоматизировать выполнение определенных действий, минимизировать избыточность кода и повысить производительность работы с данными.

---

## 1. Триггеры в PLSQL
### 1.1 Определение и назначение
Триггер — это объект базы данных, который автоматически выполняет определенные действия при возникновении событий (INSERT, UPDATE, DELETE) в таблице. Триггеры позволяют:
- Поддерживать целостность данных
- Автоматизировать логику бизнес-процессов
- Логировать изменения
- Генерировать вычисляемые значения

### 1.2 Типы триггеров
PLSQL поддерживает несколько типов триггеров:
- **BEFORE** – выполняется перед изменением данных
- **AFTER** – выполняется после изменения данных
- **INSTEAD OF** – применяется к представлениям (VIEW) для перехвата операций
- **ROW-LEVEL** – срабатывает для каждой измененной строки
- **STATEMENT-LEVEL** – выполняется один раз для всего оператора

### 1.3 Пример триггера для логирования переводов студентов
```sql
CREATE OR REPLACE TRIGGER trg_student_transfer
BEFORE UPDATE ON students
FOR EACH ROW
WHEN (NEW.status = 'переведен')
BEGIN
    INSERT INTO student_transfers (student_id, transfer_date)
    VALUES (:OLD.id, SYSDATE);
END;
/
```
Этот триггер записывает в таблицу `student_transfers` информацию о студентах, которые были переведены.

### 1.4 Пример триггера для предотвращения удаления студентов
```sql
CREATE OR REPLACE TRIGGER trg_prevent_student_delete
BEFORE DELETE ON students
FOR EACH ROW
BEGIN
    RAISE_APPLICATION_ERROR(-20001, 'Удаление студентов запрещено!');
END;
/
```
Этот триггер предотвращает удаление записей студентов, обеспечивая сохранность данных.

---

## 2. Хранимые процедуры в PLSQL
### 2.1 Определение и назначение
Хранимая процедура — это именованный блок кода, который выполняет определенные действия и может быть вызван многократно. Процедуры используются для:
- Выполнения сложных операций над данными
- Автоматизации действий
- Повышения производительности
- Снижения нагрузки на клиентские приложения

### 2.2 Создание процедуры для удаления студента
```sql
CREATE OR REPLACE PROCEDURE remove_transferred_student (p_student_id NUMBER) AS
BEGIN
    DELETE FROM students WHERE id = p_student_id AND status = 'переведен';
    COMMIT;
END;
/
```
Эта процедура удаляет студента из таблицы, если он был переведен.

### 2.3 Создание процедуры для обновления статуса студента
```sql
CREATE OR REPLACE PROCEDURE update_student_status (p_student_id NUMBER, p_new_status VARCHAR2) AS
BEGIN
    UPDATE students
    SET status = p_new_status
    WHERE id = p_student_id;
    COMMIT;
END;
/
```
### 2.4 Вызов процедуры
```sql
BEGIN
    remove_transferred_student(101);
    update_student_status(102, 'переведен');
END;
/
```

---

## 3. Функции в PLSQL
### 3.1 Определение и назначение
Функция — это объект базы данных, который возвращает значение и может использоваться в SQL-запросах. Функции применяются для:
- Выполнения вычислений
- Возвращения агрегированных значений
- Извлечения данных из таблиц

### 3.2 Создание функции для проверки статуса студента
```sql
CREATE OR REPLACE FUNCTION get_student_status (p_student_id NUMBER) RETURN VARCHAR2 IS
    v_status VARCHAR2(50);
BEGIN
    SELECT status INTO v_status FROM students WHERE id = p_student_id;
    RETURN v_status;
END;
/
```
### 3.3 Вызов функции в SQL-запросе
```sql
SELECT get_student_status(101) FROM dual;
```

---

## 4. Использование триггеров, процедур и функций в управлении студентами
Рассмотрим сценарий работы с базой данных студентов:
1. Студент подает заявку на перевод.
2. Оператор обновляет его статус в базе данных на 'переведен'.
3. Автоматически срабатывает триггер, который записывает информацию о переводе в таблицу `student_transfers`.
4. По мере необходимости вызывается процедура `remove_transferred_student`, которая удаляет запись о студенте из таблицы `students`.
5. Можно использовать функцию `get_student_status`, чтобы проверить статус любого студента.

### Пример объединенного сценария
```sql
BEGIN
    -- Обновляем статус студента
    update_student_status(201, 'переведен');
    
    -- Проверяем его статус
    DBMS_OUTPUT.PUT_LINE(get_student_status(201));
    
    -- Удаляем запись о студенте, если он переведен
    remove_transferred_student(201);
END;
/
```

---

## Заключение
PLSQL предоставляет мощные инструменты для автоматизации работы с базой данных. Триггеры помогают отслеживать изменения в таблицах, процедуры позволяют выполнять сложные операции, а функции позволяют получать вычисляемые значения, которые можно использовать в SQL-запросах. 

Использование PLSQL в управлении студенческой базой данных позволяет автоматизировать удаление записей о переведенных студентах, вести историю их перемещения и обеспечивать целостность данных. Грамотное применение этих инструментов повышает надежность и эффективность работы с базами данных.

